In [2]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from transformers import  BertTokenizer, TFAlbertModel
import implicit
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import tensorflow as tf


In [3]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
numpyArray = np.load('../Jiro/features.npy')
np.load = np_load_old
ratings_ = df = pd.DataFrame(numpyArray, columns = ['user_id','business_id','stars','text'])

In [4]:
ratingsnew = ratings_.head(10000)
# ratingsnew["stars"].astype(float).round()

for count in range(len(ratingsnew.text)):
    ratingsnew.at[count, 'text'] = " ".join(ratingsnew.at[count, 'text'])

ratingsnew['text'] = ratingsnew['text'].astype(str)
print(ratings_.head(1000))


                    user_id             business_id stars  \
0    mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   3.0   
1    OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   5.0   
2    8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   3.0   
3    _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   5.0   
4    bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   4.0   
..                      ...                     ...   ...   
995  syyKcKPFILDysHWmtka-aA  1_hDCN3iioFR3XnUr32ZtA   5.0   
996  _lgLNzpzf3qmbwySBakxEw  5RzJ2bjU8bLSaN5SuiUpYA   4.0   
997  Sh_vUlHHY2Kuj14eF8NYZQ  s1PNBO9o5jIgNd5YWUDLXQ   5.0   
998  YwMD-AVT67fmYRGxnlRSPA  alUk6OwNhofyc90NDMDY-Q   5.0   
999  WKe2b_EeLBnZ3lZV5WKYGQ  -Or44IdY51Ukd618kikmtA   4.0   

                                                  text  
0    [CLS] If you decide to eat here , just be awar...  
1    [CLS] I ' ve taken a lot of spin classes over ...  
2    [CLS] Family diner . Had the b ##uff ##et . E ...  
3    [CLS] Wow ! Yu ##mmy , different ,

C:\Users\apran\AppData\Local\Temp\ipykernel_1368\2454310699.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratingsnew['text'] = ratingsnew['text'].astype(str)


In [5]:
# df_features = df_ratings.pivot_table(
#     index = "user_id",
#     columns = "business_id",
#     values = "labels",
# ).fillna(0)
df_features = ratingsnew
df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes

C:\Users\apran\AppData\Local\Temp\ipykernel_1368\1422376793.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
C:\Users\apran\AppData\Local\Temp\ipykernel_1368\1422376793.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes


In [6]:

sparse_item_user = sparse.csr_matrix((df_features['stars'].astype('float'), (df_features['user_id'], df_features['business_id'])))
sparse_user_item = sparse.csr_matrix((df_features['stars'].astype('float'),  (df_features['business_id'],df_features['user_id'])))

In [7]:
modelRec = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')


modelRec.fit(data_conf)

c:\Users\apran\anaconda3\envs\nlp\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [8]:
df_features["labels"] = df_features.apply(lambda row: modelRec.user_factors[row['user_id']],axis=1)
df_features.head()

C:\Users\apran\AppData\Local\Temp\ipykernel_1368\3483575756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["labels"] = df_features.apply(lambda row: modelRec.user_factors[row['user_id']],axis=1)


,user_id,business_id,stars,text,labels
0,7445,2091,3.0,"[CLS] If you decide to eat here , just be awar...","[0.41187847, -0.1470871, 1.4230313, -0.985572,..."
1,3849,525,5.0,[CLS] I ' ve taken a lot of spin classes over ...,"[1.4271379, 0.5734841, 0.22492506, -0.7197593,..."
2,1395,2179,3.0,[CLS] Family diner . Had the b ##uff ##et . E ...,"[0.033039246, -0.04612458, 0.040598862, -0.001..."
3,5467,2992,5.0,"[CLS] Wow ! Yu ##mmy , different , delicious ....","[1.9146069, 0.61366564, -1.0416332, -1.0952092..."
4,5864,2555,4.0,[CLS] Cut ##e interior and owner ( ? ) gave us...,"[2.2135386, -1.1574129, 0.57346815, 0.02970468..."


In [9]:
train_df, test_df = train_test_split(df_features, test_size = 0.2)
y_train = tf.convert_to_tensor(
    train_df["labels"].tolist(), dtype=float
)

y_test = tf.convert_to_tensor(
    test_df["labels"].tolist(), dtype=float
)

# y_train = tf.convert_to_tensor(train_df["stars"],dtype = "float")
# y_test = tf.convert_to_tensor(test_df["stars"],dtype = "float")

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
albert = TFAlbertModel.from_pretrained('roberta-base')

You are using a model of type roberta to instantiate a model of type albert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at roberta-base were not used when initializing TFAlbertModel: ['roberta', 'lm_head']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFAlbertModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['albert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
x_train = tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [12]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [14]:
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = albert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(20,activation = 'linear')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [15]:
model.layers[2].trainable = False

In [16]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = MeanSquaredError()
metric = MeanSquaredError()
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = [MeanSquaredError(),MeanAbsolutePercentageError()])

In [17]:
import datetime
logdir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=2,
    batch_size=36
)

Epoch 1/2
223/223 [==============================] - 334s 1s/step - loss: 1.2559 - mean_squared_error: 1.2558 - mean_absolute_percentage_error: 13094493.0000 - val_loss: 1.1461 - val_mean_squared_error: 1.1457 - val_mean_absolute_percentage_error: 6001697.5000
Epoch 2/2
223/223 [==============================] - 349s 2s/step - loss: 1.1373 - mean_squared_error: 1.1379 - mean_absolute_percentage_error: 6990637.0000 - val_loss: 1.1451 - val_mean_squared_error: 1.1448 - val_mean_absolute_percentage_error: 5765196.0000


In [18]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

63/63 [==============================] - 64s 979ms/step


array([ 0.20051697,  0.15412249,  0.15294255,  0.21138117,  0.11549423,
        0.05137752,  0.144123  ,  0.06729294, -0.19569474,  0.07767697,
        0.09435569,  0.01840441,  0.14983043, -0.00851999,  0.10987406,
       -0.06124957,  0.24819764,  0.14339711, -0.04253792, -0.02522406],
      dtype=float32)

In [34]:
user_id = train_df['user_id'].tolist()[2]
print(user_id)

3891


In [35]:
newVector = np.array(predicted_raw[3])
print(newVector)
modelRec.user_factors[user_id]

[ 0.10295575  0.11375352  0.06930508  0.11074536  0.05934067  0.04118954
  0.01518329  0.02386541 -0.13591608  0.04363151 -0.03618512  0.00564639
  0.16458677  0.08016533  0.12169913 -0.01644412  0.1943489   0.09406001
  0.03913291 -0.04028853]


array([ 0.6539232 , -0.35636786, -1.0438573 ,  0.19457965, -0.83032054,
        1.388757  , -0.6889793 , -1.1521589 ,  0.66462904,  0.59080464,
       -0.59454584,  0.70876104, -0.06493695,  1.3636272 ,  0.9936043 ,
        0.16275223,  0.6139103 , -0.05925844,  1.4367601 ,  0.7175657 ],
      dtype=float32)

In [36]:
modelRec.recommend(user_id,data_conf[user_id], N = 20, filter_already_liked_items=False)

(array([3286, 1743, 1998, 1626, 1291, 2008, 1542, 1010, 1267,  153, 1189,
        3585, 1532, 3653, 2868, 2404, 3889,  722, 3075, 1260]),
 array([0.48533034, 0.38129288, 0.36681217, 0.3262185 , 0.3163708 ,
        0.29860687, 0.29297346, 0.2757267 , 0.27488968, 0.27380997,
        0.27124417, 0.26946998, 0.26901555, 0.2666011 , 0.25310764,
        0.2527104 , 0.24510266, 0.2438027 , 0.23644276, 0.22986583],
       dtype=float32))

In [37]:
modelRec.user_factors[user_id] = newVector
print(modelRec.recommend(user_id,data_conf[user_id], N =20))

(array([ 752, 2951, 3185, 1499, 1626, 2047,  982, 2571,  159, 3825, 1994,
        988, 2558, 1679, 3895, 1913,  321,  926, 3850,  367]), array([0.04245755, 0.04238702, 0.03885078, 0.03883074, 0.03744131,
       0.03709229, 0.0366518 , 0.03629455, 0.03434301, 0.03400378,
       0.03192267, 0.03088869, 0.03074656, 0.0306809 , 0.0304407 ,
       0.03039046, 0.0298896 , 0.02965244, 0.02959996, 0.02953715],
      dtype=float32))
